In [1]:
from owl import *
from trueskill import Rating, rate, BETA, global_env
import itertools
import math

def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

In [13]:
matches = load_json('stats/matches.json')
teams = {}
players = {}
formations = {}
for match in matches:
    vs = []
    vs_formation = []
    all_players = []
    all_formations = []
    for i, team in enumerate(match['teams']):
        team_name = match['team_names'][i]
        team_players = []
        if team_name not in teams:
            teams[team_name] = {'players': {}, 'name': team_name, 'formations': {}}
        for player_name in team:
            player_name = ','.join([player_name, team_name])
            if player_name not in players:
                players[player_name] = {'rate': Rating(), 'name': player_name}
            teams[team_name]['players'][player_name] = players[player_name]['rate']
            team_players.append(players[player_name]['rate'])
            all_players.append(player_name)
        formation_name = ','.join(sorted(team))
        if formation_name not in formations:
            formations[formation_name] = {'rate': Rating(), 'name': formation_name}
        teams[team_name]['formations'][formation_name] = formations[formation_name]['rate']
        vs.append(team_players)
        vs_formation.append([formations[formation_name]['rate']])
        all_formations.append(formation_name)
    ranks = match['ranks']
    ranks.reverse()
    new_ratings = rate(vs, ranks)
    new_ratings = list(new_ratings[0]) + list(new_ratings[1])
    for i, rating in enumerate(new_ratings):
        players[all_players[i]]['rate'] = rating
    new_ratings = rate(vs_formation, ranks)
    new_ratings = list(new_ratings[0]) + list(new_ratings[1])
    for i, rating in enumerate(new_ratings):
        formations[all_formations[i]]['rate'] = rating


In [14]:
teams.keys()

dict_keys(['SFS', 'VAL', 'SHD', 'GLA', 'DAL', 'SEO', 'LDN', 'FLA', 'PHI', 'HOU', 'BOS', 'NYE'])

In [59]:
# [(d['name'], d['rate']) for d in sorted(players.values(), key=lambda x: x['rate'].mu, reverse=True)]
def best_formation(team_name):
    return sorted(teams[team_name]['formations'].items(), key=lambda x:x[1].mu, reverse=True)[0][1]

def most_formation(team_name):
    return sorted(teams[team_name]['formations'].items(), key=lambda x:x[1].sigma, reverse=False)[0][1]

sorted(teams['SHD']['formations'].items(), key=lambda x:x[1].mu, reverse=True)

[('Altering,FiveKing,Freefeel,Xushu,mg,uNdeAD',
  trueskill.Rating(mu=28.010, sigma=4.911)),
 ('Ado,Altering,Diya,Fearless,Geguri,Sky',
  trueskill.Rating(mu=25.896, sigma=2.902)),
 ('Altering,Diya,FiveKing,Roshan,Xushu,uNdeAD',
  trueskill.Rating(mu=25.743, sigma=5.562)),
 ('Diya,FiveKing,Freefeel,Roshan,Xushu,uNdeAD',
  trueskill.Rating(mu=25.000, sigma=4.815)),
 ('Altering,Freefeel,Roshan,Xushu,mg,uNdeAD',
  trueskill.Rating(mu=25.000, sigma=8.333)),
 ('Ado,Altering,Diya,Fearless,Freefeel,Geguri',
  trueskill.Rating(mu=24.163, sigma=2.327)),
 ('Ado,Altering,Fearless,FiveKing,Xushu,mg',
  trueskill.Rating(mu=23.587, sigma=4.028)),
 ('Altering,FiveKing,Roshan,Xushu,mg,uNdeAD',
  trueskill.Rating(mu=23.317, sigma=3.835)),
 ('Ado,Daemin,Fearless,FiveKing,Freefeel,Geguri',
  trueskill.Rating(mu=22.787, sigma=7.331)),
 ('Ado,Diya,Fearless,FiveKing,Freefeel,Geguri',
  trueskill.Rating(mu=22.676, sigma=2.453)),
 ('Ado,Fearless,FiveKing,Freefeel,Xushu,mg',
  trueskill.Rating(mu=22.620, sigma

In [58]:
win_probability([most_formation('BOS')], [most_formation('SHD')])

0.9663346244262913